# Convertible Bond Strategy - Factor 0002

## 1. Import Library

In [13]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [14]:
sd = '2024-12-31'
ed = '2026-02-13'

In [15]:
sql_factor = f"""
SELECT
    date,
    instrument,
    conversion_premium_rate AS factor 
FROM cn_cbond_bar1d JOIN cn_cbond_analyze_metric USING (date, instrument)
WHERE bond_balance > 1.5 
AND remaining_days >120
AND current_yield > 0
"""

In [16]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 20
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [17]:
df_strategy = dai.query(sql_trade, filters={"date":[sd,ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2024-12-31,110815.SH,-24.815983,1.0,0.05
1,2024-12-31,110816.SH,-3.157518,2.0,0.05
2,2024-12-31,123147.SZ,-1.785591,3.0,0.05
3,2024-12-31,127032.SZ,-0.135684,4.0,0.05
4,2024-12-31,113044.SH,0.095963,5.0,0.05
...,...,...,...,...,...
5475,2026-02-13,123178.SZ,3.680376,16.0,0.05
5476,2026-02-13,113644.SH,3.752895,17.0,0.05
5477,2026-02-13,118034.SH,6.495942,18.0,0.05
5478,2026-02-13,123209.SZ,6.764766,19.0,0.05


## 3. Backtest Strategy

In [18]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 21:06:26] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 21:06:26] [info     ] 2024-12-31, 2026-02-13, , , instruments=245
[2026-02-18 21:06:26] [info     ] bigtrader module V2.2.0
[2026-02-18 21:06:26] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '110815.SH' at '2024-12-31 15:00:00'
order_target_percent: price is null for '110816.SH' at '2024-12-31 15:00:00'
order_target_percent: price is null for '110808.SH' at '2024-12-31 15:00:00'
order_target_percent: price is null for '124024.SZ' at '2024-12-31 15:00:00'
order_target_percent: price is null for '110815.SH' at '2025-01-02 15:00:00'
order_target_percent: price is null for '110816.SH' at '2025-01-02 15:00:00'
order_target_percent: price is null for '124024.SZ' at '2025-01-02 15:00:00'
order_target_percent: price is null for '110815.SH' at '2025-01-03 15:00:00'
order_target_percent: price is null for '110816.SH' at '2025-01-03 15:00:00'
order_target_percent: price is 

[2026-02-18 21:06:31] [info     ] bigtrader.v34 运行完成 [4.930s].
